# T-Maze CBFE Value Decomposition

This notebook generates the value decomposition diagrams of Fig. 8 (Sec. 6.3).

In [31]:
using Plots
using LinearAlgebra
using ForneyLab

# T-maze layout
# [2| |3]
#   | |
#   |1|
#   |4|

# Constrained Model

In [32]:
T = 2

fg_plan = FactorGraph()

u = Vector{Variable}(undef, T)
x = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

@RV x_t_min ~ Categorical(placeholder(:D_t_min, dims=(8,)))

x_k_min = x_t_min
for k=1:T
    @RV u[k]
    @RV x[k] ~ Transition(x_k_min, u[k])
    @RV y[k] ~ Transition(x[k], placeholder(:A, dims=(16,8), var_id=:A_*k))

    placeholder(u[k], :u, index=k, dims=(8,8))
    Categorical(y[k], placeholder(:C, dims=(16,), index=k, var_id=:C_*k))
    PointMassConstraint(y[k])
    
    x_k_min = x[k]
end
;

In [33]:
q_plan_constrained = PosteriorFactorization(y, [x_t_min; x], ids=[:Y, :X])
algo_plan_constrained = messagePassingAlgorithm(y, id=:Plan, free_energy=true)
code_plan_constrained = algorithmSourceCode(algo_plan_constrained, free_energy=true)
eval(Meta.parse(code_plan_constrained))
;

In [34]:
# println(code_plan_constrained) # Uncomment to inspect algorithm source code

In [35]:
function valueTermsPlan(data::Dict, marginals::Dict)

G = 0 # Information opportunity
R = 0 # Information risk
E = 0 # Extrinsic value

R += averageEnergy(Categorical, marginals[:x_t_min], ProbabilityDistribution(Multivariate, PointMass, m=data[:D_t_min]))
E -= averageEnergy(Categorical, marginals[:y_1], ProbabilityDistribution(Multivariate, PointMass, m=data[:C][1]))
E -= averageEnergy(Categorical, marginals[:y_2], ProbabilityDistribution(Multivariate, PointMass, m=data[:C][2]))
R += averageEnergy(Transition, marginals[:x_1_x_t_min], ProbabilityDistribution(MatrixVariate, PointMass, m=data[:u][1]))
G -= averageEnergy(Transition, marginals[:y_1], marginals[:x_1], ProbabilityDistribution(MatrixVariate, PointMass, m=data[:A]))
R += averageEnergy(Transition, marginals[:x_2_x_1], ProbabilityDistribution(MatrixVariate, PointMass, m=data[:u][2]))
G -= averageEnergy(Transition, marginals[:y_2], marginals[:x_2], ProbabilityDistribution(MatrixVariate, PointMass, m=data[:A]))

R -= -1*differentialEntropy(marginals[:x_1])
R -= differentialEntropy(marginals[:x_1_x_t_min])
R -= differentialEntropy(marginals[:x_2_x_1])

return (G/log(2), R/log(2), E/log(2)) # Convert to bits

end;

# Results

In [36]:
# Reward probability and utility, uncomment to select scenario
α = 0.9; c = 2.0
# α = 0.5; c = 2.0
# α = 0.9; c = 0.0
# α = 1.0; c = 2.0

include("environment.jl")
include("agent.jl")
include("update_rules.jl")
include("helpers.jl")

(A, B, C, D) = constructABCD(α, c);

## Inspect Values

In [37]:
(G, R, E, _) = evaluateValues(A, B, C, D);

In [38]:
plotResults(G, clim=(-2.0,0.0), dpi=300, highlight=maximum)

savefig("figures/information_gain_c_$(c)_a_$(α).png")

In [39]:
plotResults(R, clim=(0.0,2.0), dpi=300, highlight=minimum)

savefig("figures/information_risk_c_$(c)_a_$(α).png")

In [40]:
plotResults(E, clim=(-10.0,0.0), dpi=300, highlight=maximum)

savefig("figures/extrinsic_value_c_$(c)_a_$(α).png")